# 事前学習済みモデルとは？

たとえば、画像分類の分野では...
* 事前学習済みモデルCNN + 学習の対象NN → 画像分類完了
* まず、「犬」と「猫」を識別するためのCNNを作成し、オブジェクトを特徴ベクトルに変換する方法を学習する
     * たとえば、この辺りに縦線がある、この辺りに丸い物体がある、etc...
* CNNの部分は事前学習済みのものを利用し、NNの部分でモデルを構築すればよい
* つまり、事前学習済みモデルCNNを作成しておけば、学習の対象NNが変わるたびに事前学習をいちいち行う必要がない→効率的な解析が可能
* 事前学習済みモデル：入力オブジェクトを特徴ベクトルのようなものに変換するモデルであり、その分野の様々なタスクに共通して利用できるネットワークモデル
     * 自然言語処理の場合、入力オブジェクトは単語列、タスクへの入力データは対応する単語の埋め込み表現(分散表現)の列となる

### 事前学習済みモデルの特徴
* 様々なタスクに共通して使えるため、パワフルなものを一つ作っておけばよい→構築は大変だが、作ってしまえば便利
* 下流のタスクで必要となるラベル付きデータの量を軽減できる(転移学習)→教師ありデータを作成するコストが減る
* 下流タスクによって調整可能→fine-tuning

### BERT(Bidirectional Encoder Representations from Transformers)
* 2018年末に出現→最先端！

##### Word2Vecによる分散表現との違い
* 出力する単語の埋め込み表現は文脈依存になっている、周辺の単語との関係から埋め込み表現を作成する
     * 「私は犬が好き。」の「犬/dog」と「奴は警察の犬だ。」の「犬/spy」は語義が異なる
     * Word2Vecでは、どちらの場合でも分散表現は固定
* 事前学習済みモデルであるため、fine-tuningが可能
     * Word2Vecは辞書のようなものであるため、タスクに応じて修正不可
     * BERTでは、タスクに応じてネットワークの重みを修正可能

* transformers [HuggingFace] のインストール

In [1]:
pip install transformers[ja]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


* 日本語版BERTであるcl-tohoku/bert-base-japanese-v2を学習させる

In [2]:
from transformers import BertModel
bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-v2')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


* torchinfoのsummaryを使うとBERT内のパラメータの様子を確認できる

In [3]:
pip install torchinfo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
from torchinfo import summary
summary(bert)

Layer (type:depth-idx)                             Param #
BertModel                                          --
├─BertEmbeddings: 1-1                              --
│    └─Embedding: 2-1                              25,165,824
│    └─Embedding: 2-2                              393,216
│    └─Embedding: 2-3                              1,536
│    └─LayerNorm: 2-4                              1,536
│    └─Dropout: 2-5                                --
├─BertEncoder: 1-2                                 --
│    └─ModuleList: 2-6                             --
│    │    └─BertLayer: 3-1                         7,087,872
│    │    └─BertLayer: 3-2                         7,087,872
│    │    └─BertLayer: 3-3                         7,087,872
│    │    └─BertLayer: 3-4                         7,087,872
│    │    └─BertLayer: 3-5                         7,087,872
│    │    └─BertLayer: 3-6                         7,087,872
│    │    └─BertLayer: 3-7                         7,087,872
│    │   

* 入力単語列を単語id型に変換するためにtokenizerを作成
* 日本語BERTに対応したtokenizerを用いる

In [5]:
from transformers import BertJapaneseTokenizer
tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

* 単語分割

In [6]:
tknz.tokenize("私は犬が好き。")

['私', 'は', '犬', 'が', '好き', '。']

* 各単語を単語idに変換
* 「私」が3946、...、「。」が829に対応
* 文頭の特殊トークンである [CLS] (2)と文末の特殊トークン [SEP] (3)が加わっている

In [7]:
tknz.encode("私は犬が好き。")

[2, 3946, 897, 3549, 862, 12215, 829, 3]

* 特殊トークン非表示

In [8]:
tknz.encode("私は犬が好き。", add_special_tokens=False)

[3946, 897, 3549, 862, 12215, 829]

* Tensor型に変換してBERTに入力、unsqueeze(0)で要素が一つのバッチにする
     * 最適なパラメータを求める際に、勾配降下法を用いる
     * 勾配降下法を用いるには、学習するデータセットをいくつかのグループに分ける必要がある
     * このグループの数を「バッチ」と呼ぶ

In [9]:
import torch
x = tknz.encode("私は犬が好き。")
x = torch.LongTensor(x).unsqueeze(0)
x

tensor([[    2,  3946,   897,  3549,   862, 12215,   829,     3]])

* BERTの出力結果は変数yに入っている
* last_hidden_stateで、BERTの出力である分散表現の列を表示

In [10]:
y = bert(x)
y.last_hidden_state

tensor([[[ 1.6633e-01, -7.8501e-02,  1.1767e-03,  ...,  3.6960e-01,
           1.6008e-01, -5.5611e-01],
         [ 2.7389e-01, -2.8402e-01, -8.4956e-01,  ..., -6.4883e-01,
           3.8284e-01, -1.5853e-01],
         [-5.7850e-01,  3.8757e-01, -9.7429e-01,  ...,  1.2454e+00,
          -4.9265e-01, -3.7446e-01],
         ...,
         [ 7.1451e-01,  2.8899e-01, -5.4993e-01,  ...,  1.0793e-01,
          -1.8923e+00, -8.3096e-01],
         [ 3.0506e-01, -7.4390e-01, -6.6757e-01,  ...,  2.6773e-01,
          -9.7722e-01, -8.8383e-01],
         [ 1.9436e-01, -8.5321e+00, -1.4069e-01,  ...,  7.1000e-03,
          -9.3125e-02, -5.4594e-01]]], grad_fn=<NativeLayerNormBackward0>)

* last_hidden_stateの形状は[バッチサイズ, 単語列の長さ, 単語の次元数]で表される

In [11]:
y.last_hidden_state.shape

torch.Size([1, 8, 768])

* [CLS] 「私」「は」「犬」「が」「好き」「。」[SEP] の4番目(0から始まる)の埋め込み表現(犬)を表示

In [12]:
y.last_hidden_state[0][3]  #入力文内の4番目の単語「犬」の埋め込み表現

tensor([ 8.5998e-01,  1.2333e-01, -4.1867e-01,  1.2998e-01,  1.5049e-01,
        -1.1773e+00, -8.8095e-01,  3.9046e-01,  1.1116e-02,  5.0804e-02,
         6.2080e-01, -1.4059e+00, -1.5116e+00, -1.0155e+00, -5.3455e-02,
         3.3133e-01, -1.2388e+00, -1.4632e+00,  3.0444e+00, -1.0437e+00,
        -4.2564e-01,  2.7795e-01, -9.6374e-01, -6.7335e-01, -7.0273e-01,
        -2.0534e-01, -3.4195e-02,  1.0376e+00,  3.1639e-01, -9.4961e-01,
        -4.9169e-02,  3.3885e-01,  1.7551e+00,  3.5499e-01,  1.4980e+00,
         7.2804e-02,  4.8653e-01, -2.7875e-01, -4.2737e-01,  3.1235e-01,
         7.8298e-01,  2.4274e-01, -7.7656e-01, -1.8049e-01,  1.1966e+00,
        -5.4871e-02,  1.1559e+00,  9.0174e-01, -5.0762e-01, -9.7665e-01,
         2.4764e-01, -1.2593e-01, -4.3765e+00, -5.8042e-01,  6.3760e-01,
        -1.8688e+00,  2.2698e-01, -2.3121e-01,  4.7407e-01, -2.4936e-01,
         1.9515e-01, -8.7563e-01, -1.8446e-01, -7.8302e-01,  1.0440e+00,
        -5.5363e-01, -2.9012e-01, -1.5317e+00,  3.8

### BERT内部の処理
* Transformer：BERTのアーキテクチャの一種
     * 2017年に発表されたニューラル機械翻訳モデル
     * 大きくEncoderとDecoderから構成されている
     * Encoder = Embedding + Positional Encoding
     * TransformerにおけるEncoderをBERTと呼ぶ

##### BERTの構造     
* BERT = BertEmbeddings(Embedding) + BertEncoder
* BertEmbeddings内のPositional Encodingという部分で単語を分散表現に変換
* この分散表現を12個のBertLayer(まとめてBertEncoderと呼ぶ)を使って少しずつ修正している

##### BertEmbeddings
* Position Embeddings：単語の位置(Position)に関する分散表現(Embeddings)
* Segment Embeddings：1文目の単語か2文目の単語かを表す埋め込み表現のベクトル
* BertEmbeddings = 単語の分散表現ベクトル + Position Embeddings + Segment Embeddings

##### BertLayer
* BertLayer：BertEmbeddingsが入力される部分
* この入力がMulti-Head Attentionに渡され、単語の埋め込み表現列が出力される
* ここで得られた出力に対して、残差接続を行い、その結果に対してLayer Normalizationの処理を行う  
* 残差接続：Multi-Head Attentionの出力にMulti-Head Attentionの入力を足す処理  
* Layer Normalization：ニューラルネットワークの表現力の維持、学習の安定化、汎化性能の向上等を行う操作→深層学習が成功しやすくなるらしい
* その結果を線形変換し、再び残差接続とLayer Normalizationを行う

##### Multi-Head Attention
* Multi-Head Attention：Self-Attentionのモデルを並列で行っている構造
* BERTの核となる部分
* Self-Attention：ある1文の単語だけを使って計算された単語間の関連度スコア
* Attention(注意機構)：入力オブジェクトの各所に処理のための重みを付け、それらを統合して何らかの処理をする機構
* 入力単語(query)が学習された単語辞書内に無かったとしても、辞書内の単語のkeyとvalueに対して重みづけした情報を基に出力が可能となる
* 重みづけの操作の中で、辞書内の単語のvalueが更新されていく...

* より詳しい説明は https://crystal-method.com/topics/transformer-2/ へ！

# BERTによる文書分類

* 文章のラベルデータからBERTモデルを用いて文書分類を行うパッケージ
* 最終的な出力は分類されたクラス！

In [13]:
from transformers import BertForSequenceClassification

* 日本語版BERTであるcl-tohoku/bert-base-japanese-v2を学習させる
* num_labelsで分類するラベルの数を指定(デフォルトはnum_labels=2で2クラス分類器である)

In [14]:
model = BertForSequenceClassification.from_pretrained(
    'cl-tohoku/bert-base-japanese-v2',
    num_labels = 3
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [15]:
d = """
私は犬が好きです。一般に動物が好きです。
言葉が使える動物がいたら楽しいと思います。
"""

* モデリングのために、文書dをBERTへの入力の形である単語id列に変換
* 日本語の文書であるため、BertJapaneseTokenizerを用いて事前学習済みモデルを学習させる
* xは文書dを単語id列に変換し、モデリングのために必要な形に変換したもの

In [16]:
import torch
from transformers import BertJapaneseTokenizer
tknz = BertJapaneseTokenizer.from_pretrained(
    'cl-tohoku/bert-base-japanese-v2')
x = tknz.encode(d)  #
x = torch.LongTensor(x).unsqueeze(0)
y = model(x)

* logitsは文書がどのラベルに属する可能性が高いかを確率で示したlogit値
* [バッチサイズ, ラベル数]

In [17]:
y.logits.shape

torch.Size([1, 3])

* logit値の出力
* 今回は第3のラベルが最大値であるため、第3のラベルに分類されると推定する

In [18]:
y.logits[0]

tensor([ 0.1634,  0.2196, -0.0378], grad_fn=<SelectBackward0>)

* 最適化関数を設定する
* 損失関数の値に応じて、SGD(確率的勾配降下法)によってパラメータ更新が可能
* params：更新したいパラメータを渡す
* lr：学習率

In [19]:
import torch.optim as optim
opt = optim.SGD([{'params':model.parameters(), 'lr':0.01}])

* 文書dの正解ラベルを1とするとき、正解ラベルのデータをgaとする

In [20]:
ga = torch.LongTensor([1]).unsqueeze(0)

* 正解ラベルのデータgaを与えてモデリングすることで、最適化関数optによってパラメータ更新が可能

In [21]:
y = model(x, labels=ga)  #学習

* 損失値loss
* stepはパラメータ更新の処理
* 以下の処理を繰り返すことで、学習の処理を行うことができる

In [22]:
loss = y.loss
opt.zero_grad()
loss.backward()
opt.step()

* 学習終了後のモデルを、mymodel.binとして保存

In [23]:
torch.save(model.state_dict(), 'mymodel.bin')

* 保存したモデルをロードするために、保存したモデルの定義を行う
* configファイルを保存

In [24]:
import pickle


with open('myconfig.pkl', 'bw') as fw:
    pickle.dump(model.config, fw)

* 保存したモデルのロード

In [25]:
with open('myconfig.pkl', 'br') as f:
    myconfig = pickle.load(f)
    mymodel = BertForSequenceClassification(config=myconfig)
    mymodel.load_state_dict(torch.load('mymodel.bin'))

# BERTによる系列ラベリング

* 系列ラベリング：文章をtokenizeし、それぞれのtokenにラベルを付与する方法→BertForTokenClassificationを利用
* BertForTokenClassification→入力となるデータ系列の各データにラベルを付与するタスクに適している
* 各tokenに対する各ラベルの確率からビタビアルゴリズムを利用して最終的なラベルを決定
     * ビタビアルゴリズム：観測された事象系列を結果として生じる隠された状態の最も尤もらしい並びを探す動的計画法アルゴリズム(wiki)

In [26]:
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
                 'cl-tohoku/bert-base-japanese-v2',
                 num_labels = 9)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

In [27]:
s = "田中さんは茨城大学の学生です。"

In [28]:
import torch
from transformers import BertJapaneseTokenizer
tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')
tknz.tokenize(s)

['田中', 'さん', 'は', '茨城', '大学', 'の', '学生', 'です', '。']

* 文章sを単語id列に変換する

In [29]:
x = tknz.encode(s)
x

[2, 13026, 11689, 897, 14121, 11188, 896, 12229, 12461, 829, 3]

* モデリング

In [30]:
x = torch.LongTensor(x).unsqueeze(0)
y = model(x)

* [バッチサイズ, 単語数, ラベル数]
* 文章sは [CLS] 「田中」「さん」「は」「茨城」「大学」「の」「学生」「です」「。」 [SEP] の11単語から成る

In [31]:
y.logits.shape

torch.Size([1, 11, 9])

* 各単語に対する各ラベルのlogit値
* y.logits [0] [i] [j] でi番目の単語に対するj番目のラベルのlogit値が得られる

In [32]:
y.logits[0].shape

torch.Size([11, 9])

* 最適化関数を設定

In [33]:
import torch.optim as optim
opt = optim.SGD([{'params':model.parameters(), 'lr':0.01}])

* ラベル名とラベルidの対応(たとえば、固有の組織名を表すラベルB-ORGのラベルidを3と設定)→テキスト参照
* 正解のラベル列のデータgaを作成

In [34]:
ga = torch.LongTensor([0, 1, 0, 0, 3, 7, 0, 0, 0, 0, 0]).unsqueeze(0)

* 先程と同様に正解ラベル列のデータgaを与えたモデルを作成し、学習を行う

In [35]:
y = model(x, labels=ga)

In [36]:
loss = y.loss
opt.zero_grad()
loss.backward()
opt.step()

# Pipelineによるタスクの推論
* pipeline：各種タスクに対する事前学習済みモデルを利用して、そのタスクの推論処理を行ってくれるコマンド

### 自然言語処理におけるタスクの種類
* conversational：対話
* feature-extraction：特徴抽出
* fill-mask：マスク指定(隠す単語を指定して、推論を行わせる)
* image-classification：画像識別
* question-answering：質問応答
* table-question-answering：表内容からの質問応答
* text2text-generation：翻訳、要約、質問応答
* text-classification / sentiment-analysis：評判分析
* text-generation：テキスト生成
* token-classification / ner：固有表現抽出
* translation：翻訳
* translation_xx_to_yy：XX_YY翻訳
* summarization：要約
* zero-shot-classification：Zero-shot分類

In [37]:
from transformers import pipeline

### 評判分析
* 入力文が肯定的か否定的かを判定するタスク
* 文書分類と基本的に同じ
* レビュー分析とかに使える？

In [38]:
net = pipeline('text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [39]:
text1 = 'This book is very interesting.'
net(text1)

[{'label': 'POSITIVE', 'score': 0.9998513460159302}]

In [40]:
text2 = 'This book has some interesting parts.'
net(text2)

[{'label': 'POSITIVE', 'score': 0.9996337890625}]

* roberta-large-mnli：事前学習済みモデルRoBERTa-largeとデータセットMNLIを利用して学習されたモデル

In [41]:
net2 = pipeline('text-classification', model='roberta-large-mnli')  #positive, neutral, negativeで判定

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
net2(text2)

[{'label': 'NEUTRAL', 'score': 0.6103535294532776}]

* daigo/bert-base-japanese-sentiment：日本語の評判分析モデルの１つ

In [43]:
# net3 = pipeline('text-classification',
#                model='daigo/bert-base-japanese-sentiment')
# net3("この犬は本当にお利口さんだ。")

### 固有表現抽出

In [44]:
net = pipeline('ner')
text = 'Mr.Tanaka is a student at Ibaraki University.'
net(text)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'entity': 'I-PER',
  'score': 0.99895644,
  'index': 3,
  'word': 'Tanaka',
  'start': 3,
  'end': 9},
 {'entity': 'I-ORG',
  'score': 0.9990582,
  'index': 8,
  'word': 'I',
  'start': 26,
  'end': 27},
 {'entity': 'I-ORG',
  'score': 0.99033654,
  'index': 9,
  'word': '##bara',
  'start': 27,
  'end': 31},
 {'entity': 'I-ORG',
  'score': 0.9977101,
  'index': 10,
  'word': '##ki',
  'start': 31,
  'end': 33},
 {'entity': 'I-ORG',
  'score': 0.9953762,
  'index': 11,
  'word': 'University',
  'start': 34,
  'end': 44}]

* dslim/bert-base-NER：BERTとCoNLL-2003を使って学習されたモデル

In [45]:
# net2 = pipeline('ner',model='dslim/bert-base-NER')
# net2(text)

### 要約
* デフォルトの事前学習済みモデル：sshleifer/distilbart-cnn-12-6
* BART(テキスト生成に利用される事前学習済みモデル)とデータセットCNN/DailyMailDatasetを利用して学習されたモデル

In [46]:
net = pipeline('summarization')
doc = """We introduce a new language representation 
model called BERT, which stands for Bidirectional Encoder 
Representations from Transformers. Unlike recent language 
representation models, BERT is designed to pre-train deep 
bidirectional representations from unlabeled text by 
jointly conditioning on both left and right context in all 
layers. As a result, the pre-trained BERT model can be 
fine-tuned with just one additional output layer to create 
state-of-the-art models for a wide range of tasks, such as 
question answering and language inference, without 
substantial task-specific architecture modifications. BERT 
is conceptually simple and empirically powerful. It obtains 
new state-of-the-art results on eleven natural language 
processing tasks, including pushing the GLUE score to 80.5% 
(7.7% point absolute improvement), MultiNLI accuracy to 
86.7% (4.6% absolute improvement), SQuAD v1.1 question 
answering Test F1 to 93.2 (1.5 point absolute improvement) 
and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute 
improvement)."""
net(doc, max_length=50, min_length=20)  #要約の長さ指定

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' BERT is designed to pre-train deep bidirectional representations from unlabeled text by conditioning on both left and right context in all layers . The pre-trained BERT model can be fine-tuned with just one'}]

### 質問応答
* デフォルトの事前学習済みモデル：distilbert-base-cased-distilled-squad
* 事前学習済みモデルDistilBERTとデータセットSQuADを利用して学習されたモデル

In [47]:
net = pipeline('question-answering')
q = "How many tasks did BERT get SOTA on?"
net(context=doc, question=q)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.7516631484031677, 'start': 716, 'end': 722, 'answer': 'eleven'}

### テキスト生成
* デフォルトの事前学習済みモデル：GPT-2
* 途中までの文章を与え、それらしい文を生成する

In [48]:
net = pipeline('text-generation')
net("In this paper, we propose a new")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\snaka\AppData\Roaming\Python\Python39\site-packages\transformers\generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'In this paper, we propose a new hypothesis about the relationship between the amount of time each person spent in the family, by age, sex, occupation, and race, and their contribution to the average social life of children and adults, using a large'}]

### Zero-shot文書分類
* デフォルトの事前学習済みモデル：facebook/bart-large-mnli
* 訓練データを用いずに文書分類を行う
* ラベル自体が言語で表現されていることを利用
* ラベルが埋め込まれるベクトル空間と文書が埋め込まれるベクトル空間が同一であれば、ラベルと文書間の距離を測ることができる
* 距離が最小のラベルに文書を分類する

In [49]:
net = pipeline('zero-shot-classification')
doc = "Ichiro is the best baseball player ever."
labels = ["politics", "economics", "sports"]
net(doc, labels)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'Ichiro is the best baseball player ever.',
 'labels': ['sports', 'economics', 'politics'],
 'scores': [0.9978312849998474, 0.0013916805619373918, 0.0007770126685500145]}

In [50]:
doc = "My dog is really smart."
labels = ["positive", "negative"]
net(doc, labels)

{'sequence': 'My dog is really smart.',
 'labels': ['positive', 'negative'],
 'scores': [0.9315671920776367, 0.06843287497758865]}